In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, auc, roc_curve, f1_score, roc_auc_score

In [15]:
df = pd.read_csv('../data/creditcard.csv')

In [16]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [17]:
# 'Class' is target variable
X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [18]:


def scale_train_test(X_train, X_test):
    scaler = MinMaxScaler()
    # Fit on the training data and transform both training and test sets
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert numpy arrays back to dataframes
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    return X_train_scaled, X_test_scaled

# Usage
X_train_scaled, X_test_scaled = scale_train_test(X_train, X_test)


In [19]:
ros = RandomOverSampler(random_state=1)
X_train_oversampled, y_train_oversampled = ros.fit_resample(X_train_scaled, y_train)
y_train_oversampled.value_counts()

Class
0    227440
1    227440
Name: count, dtype: int64

In [20]:
# initialize random forest
rf_clf = RandomForestClassifier(random_state=1)

In [21]:
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)

In [22]:
# ROC curve 
y_pred_proba_rf = rf_clf.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

In [23]:
# baseline evaluation
print("Random Forest - Baseline Model Performance")
print(f"Precision: {precision_score(y_test, y_pred_rf):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.2f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf):.2f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba_rf):.2f}")

Random Forest - Baseline Model Performance
Precision: 0.93
Recall: 0.77
F1-Score: 0.84
ROC AUC: 0.94


In [24]:
print(f"The training score is {rf_clf.score(X_train_oversampled, y_train_oversampled)}")
print(f"The testing score is {rf_clf.score(X_test_scaled, y_test)}")

print("Classification Report for Random Forest Model:")
print(classification_report(y_test, y_pred_rf))

The training score is 0.5
The testing score is 0.9984726659878516
Classification Report for Random Forest Model:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56875
           1       0.93      0.77      0.84        87

    accuracy                           1.00     56962
   macro avg       0.97      0.89      0.92     56962
weighted avg       1.00      1.00      1.00     56962



In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, f1_score, auc, roc_curve

# Assuming X_train_scaled, y_train_oversampled, X_test_scaled, and y_test are already defined

# Define parameter grid
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the grid search
grid_rf = GridSearchCV(RandomForestClassifier(random_state=1), param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

# Fit the grid search to the data
grid_rf.fit(X_train_oversampled, y_train_oversampled)

# Get the best model
best_rf = grid_rf.best_estimator_

# Make predictions with the optimized model
optimized_y_pred_rf = best_rf.predict(X_test_scaled)
optimized_y_pred_proba_rf = best_rf.predict_proba(X_test_scaled)[:, 1]
optimized_fpr_rf, optimized_tpr_rf, _ = roc_curve(y_test, optimized_y_pred_proba_rf)
optimized_roc_auc_rf = auc(optimized_fpr_rf, optimized_tpr_rf)

# Print the performance of the baseline model
print("Random Forest - Baseline Model Performance:")
print(f"Precision: {precision_score(y_test, y_pred_rf):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.2f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf):.2f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba_rf):.2f}")

# Print the performance of the optimized model
print("\nRandom Forest - Optimized Model Performance:")
print(f"Optimized Precision: {precision_score(y_test, optimized_y_pred_rf):.2f}")
print(f"Optimized Recall: {recall_score(y_test, optimized_y_pred_rf):.2f}")
print(f"Optimized F1-Score: {f1_score(y_test, optimized_y_pred_rf):.2f}")
print(f"Optimized ROC AUC: {roc_auc_score(y_test, optimized_y_pred_proba_rf):.2f}")

# Training and testing scores comparison
print(f"\nBaseline Training Score: {rf_clf.score(X_train_scaled, y_train)}")
print(f"Baseline Testing Score: {rf_clf.score(X_test_scaled, y_test)}")
print(f"Optimized Training Score: {best_rf.score(X_train_oversampled, y_train_oversampled)}")
print(f"Optimized Testing Score: {best_rf.score(X_test_scaled, y_test)}")


KeyboardInterrupt: 